In [15]:
from langchain_community.llms import Ollama

llm = Ollama(model="mistral")

In [1]:
from qdrant_client import QdrantClient

client = QdrantClient(host="localhost", port=6333)

In [3]:
query = 'किस तरह के किरदार और कहानी तत्व रचनाकारों और फिल्म निर्माताओं को आकर्षित करते हैं?'

In [4]:
import fasttext as ft
# Loding model for Hindi.
embed_model = ft.load_model('wiki.hi.bin')

In [9]:
from qdrant_client.models import Filter, FieldCondition, Range

# hits = client.search(
#    collection_name="my_collection",
#    query_vector=embed_model.get_sentence_vector(query).tolist(),
#    query_filter=Filter(
#       must=[  # These conditions are required for search results
#             FieldCondition(
#                key='rand_number',  # Condition based on values of `rand_number` field.
#                range=Range(
#                   gte=3  # Select only those results where `rand_number` >= 3
#                )
#             )
#       ]
#    ),
#    limit=5  # Return 5 closest points
# )



hits = client.search(
collection_name="my_collection",
query_vector= embed_model.get_sentence_vector(query).tolist(),
limit=1,
)

In [19]:
llm.invoke(query+'you can only reply in hindi, english not allowed')

' To attract screenwriters and filmmakers, production houses and studios use various strategies. Some common methods are:\n\n1. Offering compelling stories and scripts: Production houses create a pool of high-quality scripts that cater to the current market trends and audience preferences. They often collaborate with established writers or provide opportunities for new talent to write engaging stories.\n\n2. Providing adequate resources and facilities: A well-equipped studio, modern technology, experienced technicians, and ample budget are some essential factors that can attract skilled professionals.\n\n3. Famous directors and actors: The involvement of popular directors and actors in a project can create excitement and generate interest among screenwriters and filmmakers.\n\n4. Lucrative deals and packages: Production houses may offer attractive remuneration packages, royalties, and profit-sharing arrangements to attract talent.\n\n5. Creative freedom and collaboration: Allowing arti

In [ ]:
def generate_text(question):
 # Searching for relevant hits in the 'speech_collection'
 hits = client.search(
 collection_name="my_collection",
 query_vector= embed_model.get_sentence_vector(question).tolist(),
 limit=10,
 )
 # Creating context from the hits
 context = ''
 for hit in hits:
    context += hit.payload['page_content'] + '\n'
 # Constructing the prompt
 prompt = f"""<s>[INST] आप एक सम्मानीय सहायक हैं। आपका काम नीचे दिए गए संदर्भ से प्रश्नों का उत्तर देना है।
 संदर्भ: {context}
 प्रश्न: {question} [/INST] </s>
 """
 # Generating text using the GPT model
 sequences = pipeline(
 prompt,
 do_sample=True,
 temperature=0.7,
 top_k=50,
 top_p=0.95,
 num_return_sequences=1,
 )
 return sequences[0]['generated_text']

In [11]:
chat_history = []

In [ ]:
       # Create the custom chain
       if llm is not None and db is not None:   
              chain=ConversationalRetrievalChain.from_llm(llm=llm,retriever=retriever)	
       else:     
              print("LLM or Vector Database not initialized")